# Import bibliotek i danych

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import warnings
import numpy as np
import pickle
import typing
warnings.filterwarnings("ignore")
SEED = 17

# Ładowanie danych

In [2]:
data=pd.read_csv("../data/data_eda.csv")
data=data.drop(columns=['Unnamed: 0'])
features=data.columns.tolist()
features.remove('stars')
target='stars'
data

,pages,stars,reviews,series,mix,character,plot,funny,lighthearted,emotional,...,author_stars,Fiction,Nonfiction,Literary,Fantasy,Crime,Social,Children,Romans,Realism
0,273,4.00,2017,0,0.44,0.51,0.02,0.27,0.37,0.91,...,4.305000,1,1,0,0,0,1,0,1,1
1,302,3.78,7330,0,0.39,0.42,0.17,0.03,0.01,0.18,...,3.670000,1,0,0,0,1,0,0,0,0
2,400,4.15,16761,0,0.51,0.39,0.08,0.02,0.01,0.88,...,0.000000,1,0,1,0,0,0,0,0,0
3,459,4.16,2128,1,0.48,0.10,0.40,0.04,0.02,0.07,...,0.000000,1,0,0,1,0,0,0,0,0
4,160,3.65,6634,1,0.28,0.16,0.54,0.92,0.73,0.00,...,4.115000,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,432,4.15,30643,0,0.48,0.05,0.46,0.00,0.00,0.40,...,3.856667,1,0,0,1,1,0,0,0,0
6926,352,3.62,1058,0,0.55,0.13,0.30,0.15,0.10,0.25,...,3.700000,1,0,0,1,0,1,0,0,0
6927,535,3.88,30975,1,0.45,0.08,0.45,0.14,0.19,0.31,...,3.870000,1,0,0,1,0,0,1,0,0
6928,472,3.88,5914,1,0.64,0.12,0.22,0.07,0.00,0.36,...,3.660000,1,0,0,1,0,0,1,0,0


$\text{Podział danych na zbiór treningowy i testowy}$

In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=SEED)

In [4]:
def perform_cv(X: pd.DataFrame, y: pd.Series, algorithm: typing.Any, cv: typing.Any = KFold(n_splits=5, shuffle=True, random_state=SEED), metric: typing.Any = mean_squared_error) -> typing.List[float]:
    """
    Perform cross-validation and return list of scores
    
    Args:
        X (pd.DataFrame): input data
        y (pd.Series): target data
        algorithm (typing.Any): algorithm to use for training and prediction
        cv (typing.Any): cross-validation strategy
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.List[float]: list of scores in order: train_scores, validation_scores
    """
    train_scores = []
    validation_scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        algorithm.fit(X_train, y_train)
        y_train_pred = algorithm.predict(X_train)
        y_val_pred = algorithm.predict(X_val)
        train_scores.append(metric(y_train, y_train_pred, squared=False))
        validation_scores.append(metric(y_val, y_val_pred, squared=False))
    return np.mean(train_scores), np.mean(validation_scores)

def evaluation(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, algorithm: typing.Any, metric: typing.Any = mean_squared_error) -> typing.Tuple[float, float, np.ndarray]:
    """
    Train the algorithm on the train data and evaluate on the train and test data
    
    Args:
        X_train (pd.DataFrame): input train data
        y_train (pd.Series): target train data
        X_test (pd.DataFrame): input test data
        y_test (pd.Series): target test data
        algorithm (typing.Any): algorithm to use for training and prediction
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.Tuple[float, float, np.ndarray]: train_score, test_score, predictions on test data
    """
    algorithm.fit(X_train, y_train)
    y_train_pred = algorithm.predict(X_train)
    y_test_pred = algorithm.predict(X_test)
    train_results = metric(y_train, y_train_pred, squared=False)
    test_results = metric(y_test, y_test_pred, squared=False)
    return train_results, test_results, y_test_pred

### Najbardziej podstawowy model (bez feature engineeringu i interakcji):

In [5]:
wzor = 'stars~' + '+'.join(features)
mod = smf.ols(formula = wzor, data = train_data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.461
Method:                 Least Squares   F-statistic:                     153.9
Date:                Fri, 24 May 2024   Prob (F-statistic):               0.00
Time:                        20:01:50   Log-Likelihood:                 512.46
No. Observations:                5544   AIC:                            -960.9
Df Residuals:                    5512   BIC:                            -749.1
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.0217      0.036     84.278      0.000       2.951       3.092
pages         -2.24e-05   2.22e-05     -1.008      0.314    -6.6e-05    2.12e-05
reviews       1.023e-06   1.23e-07      8.348      0.000    7.83e-07    1.26e-06
series           0.0708      0.008      8.948      0.000       0.055       0.086
mix              0.0731      0.033      2.238      0.025       0.009       0.137
character       -0.0894      0.027     -3.357      0.001      -0.142      -0.037
plot            -0.1755      0.038     -4.609      0.000      -0.250      -0.101
funny            0.4076      0.021     19.274      0.000       0.366       0.449
lighthearted    -0.0356      0.036     -0.987      0.324      -0.106       0.035
emotional        0.3759      0.028     13.635      0.000       0.322       0.430
hopeful          0.2907      0.039      7.535      0.000       0.215       0.366
inspiring        0.2729      0.036      7.544      0.000       0.202       0.344
relaxing         0.7907      0.075     10.521      0.000       0.643       0.938
tense            0.2778      0.033      8.393      0.000       0.213       0.343
sad              0.2105      0.029      7.289      0.000       0.154       0.267
reflective      -0.0111      0.028     -0.403      0.687      -0.065       0.043
adventurous      0.1818      0.019      9.746      0.000       0.145       0.218
challenging      0.4286      0.030     14.496      0.000       0.371       0.487
informative      0.4146      0.025     16.538      0.000       0.365       0.464
mysterious       0.1753      0.020      8.930      0.000       0.137       0.214
dark            -0.0036      0.022     -0.169      0.866      -0.046       0.039
author_count     0.0025      0.000      5.206      0.000       0.002       0.003
author_stars     0.0187      0.002     10.596      0.000       0.015       0.022
Fiction          0.0786      0.032      2.461      0.014       0.016       0.141
Nonfiction       0.1668      0.017      9.681      0.000       0.133       0.201
Literary         0.0280      0.008      3.437      0.001       0.012       0.044
Fantasy          0.0242      0.010      2.364      0.018       0.004       0.044
Crime            0.0053      0.012      0.436      0.663      -0.018       0.029
Social           0.0530      0.009      6.132      0.000       0.036       0.070
Children        -0.0269      0.009     -3.148      0.002      -0.044      -0.010
Romans          -0.0634      0.010     -6.224      0.000      -0.083      -0.043
Realism         -0.0499      0.009     -5.621      0.000      -0.067      -0.033
==============================================================================
Omnibus:                      206.478   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              306.045
Skew:      

In [6]:
model = LinearRegression(fit_intercept=True)
train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.2204369364743511
Validation RMSE: 0.2220773168372384


## Model z interakcjami

In [7]:
data_interactions=pd.read_csv("../data/data_interactions.csv")
data_interactions=data_interactions.drop(columns=['Unnamed: 0'])
features_interactions=data_interactions.columns.tolist()
features_interactions.remove('stars')
train_data_interactions, test_data_interactions = train_test_split(data_interactions, test_size=0.2, random_state=SEED)
model = LinearRegression(fit_intercept=True)
train_score, validation_score = perform_cv(train_data_interactions[features_interactions], train_data_interactions[target], model)
print("Train RMSE:", train_score)
print("Validation RMSE:", validation_score)

Train RMSE: 0.1747907121971766
Validation RMSE: 0.20180332312384489


$\text{Uzyskano znaczną poprawę wyników względem modelu bazowego.}$

## Model z transformacjami zmiennych

In [8]:
data_transformations=pd.read_csv("../data/data_feature_engineering.csv")
data_transformations=data_transformations.drop(columns=['Unnamed: 0'])
features_transformations=data_transformations.columns.tolist()
features_transformations.remove('stars')
train_data_transformations, test_data_transformations = train_test_split(data_transformations, test_size=0.2, random_state=SEED)
model = LinearRegression(fit_intercept=True)
train_score, validation_score = perform_cv(train_data_transformations[features_transformations], train_data_transformations[target], model)
print("Train RMSE:", train_score)
print("Validation RMSE:", validation_score)

Train RMSE: 0.17623904378123958
Validation RMSE: 0.20184096486309838


$\text{Różnica między wynikami modelu z interakcjami a modelem z interakcjami oraz transformacjami zmiennych jest niewielka, ale na korzyść pierwszego.}$<p>
$\text{Tym samym, zdecydowano się w dalszym etapie analizy wykorzystać model z interakcjami.}$

## Usunięcie statystycznie nieistotnych zmiennych

$\text{Ponieważ aktualnie zmiennych objaśniających jest 496, to warto sprawdzić, czy usunięcie pewnych zmiennych nie pogorszy wyników modelu.}$<p>
$\text{Poniżej zweryfikowano, wpływ usunięcie statystycznie nieistotnych zmiennych na wyniki modelu.}$

In [9]:
best_score = validation_score
features_to_remove = []
while True:
    wzor = 'stars~' + '+'.join([feature for feature in features_interactions if feature not in features_to_remove])
    mod = smf.ols(formula = wzor, data = train_data_interactions)
    res = mod.fit()
    p_values = res.pvalues
    max_p_value = max([p_values[feature] for feature in p_values.index if feature not in features_to_remove])
    if max_p_value > 0.1:
        feature_to_remove = [feature for feature in p_values.index if p_values[feature] == max_p_value][0].replace(':', '*')
        features_to_remove.append(feature_to_remove)
        model = LinearRegression(fit_intercept=True)
        train_score, validation_score = perform_cv(train_data_interactions[[feature for feature in features_interactions if feature not in features_to_remove]], train_data_interactions[target], model)
        if validation_score < best_score:
            best_score = validation_score
            best_features = [feature for feature in features_interactions if feature not in features_to_remove]
    else:
        break

Wyniki dla modelu po usunięciu statystycznie nieistotnych zmiennych

In [10]:
print("Liczba zmiennych przed usunięciem:", len(features_interactions))
print("Liczba zmiennych po usunięciu:", len(best_features))
model = LinearRegression(fit_intercept=True)
train_score, validation_score = perform_cv(train_data_interactions[best_features], train_data_interactions[target], model)
print("Train RMSE:", train_score)
print("Validation RMSE:", validation_score)

Liczba zmiennych przed usunięciem: 496
Liczba zmiennych po usunięciu: 236
Train RMSE: 0.17865755311349904
Validation RMSE: 0.18955964171300735


$\text{Wyniki uległy dosyć wyraźnej poprawie po usunięciu statystycznie nieistotnych zmiennych.}$

## VIF

$\text{W ograniczonym zbiorze danych w dalszym ciągu znajduje się 236 zmiennych objaśniających.}$<p>
$\text{W przypadku regresji liniowej duża liczba ważnym elementem jest współczynnik VIF mierzący współliniowość zmiennych.}$<p>
$\text{Poniżej przedstawiono wyniki dla współczynnika VIF dla zmiennych objaśniających.}$

In [11]:
# the independent variables set 
X = train_data_interactions[best_features]
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 
vif_data.sort_values(by=["VIF"], ascending=False).head(50)

,feature,VIF
92,plot*Fiction,inf
2,character,inf
3,plot,inf
68,mix*Fiction,inf
80,character*Fiction,inf
1,mix,inf
15,author_count,1316.951040
218,author_count*author_stars,1125.206684
16,Fiction,1069.422535
6,emotional,721.793010


$\text{Usuniemy wszystkie zmienne, dla których odnotowano wartość współczynnika VIF powyżej 10 i zobaczymy, jak wpłynie to na wyniki modelu.}$

In [12]:
vif_features_to_remove = vif_data[vif_data["VIF"] > 10]["feature"].tolist()
features_after_vif = [feature for feature in best_features if feature not in vif_features_to_remove]
model = LinearRegression(fit_intercept=True)
train_score, validation_score = perform_cv(train_data_interactions[features_after_vif], train_data_interactions[target], model)
print("Train RMSE:", train_score)
print("Validation RMSE:", validation_score)

Train RMSE: 0.25042615428796794
Validation RMSE: 0.25482944326694995


$\text{Usunięcie zmiennych o współczynniku VIF powyżej 10 wpłynęłol na znaczne pogorszenie wyników modelu.}$<p>
$\text{W związku z tym, zdecydowano się na pozostawienie wszystkich zmiennych objaśniających w modelu.}$

In [13]:
wzor = 'stars~' + '+'.join(best_features)
mod = smf.ols(formula = wzor, data = train_data_interactions)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.630
Method:                 Least Squares   F-statistic:                     39.60
Date:                Fri, 24 May 2024   Prob (F-statistic):               0.00
Time:                        20:14:38   Log-Likelihood:                 1661.3
No. Observations:                5544   AIC:                            -2833.
Df Residuals:                    5299   BIC:                            -1211.
Df Model:                         244                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     2.5311      0.134     18.845      0.000       2.268       2.794
series                        0.0972      0.033      2.912      0.004       0.032       0.163
mix                          -0.6745      0.101     -6.673      0.000      -0.873      -0.476
character                    -0.1385      0.055     -2.518      0.012      -0.246      -0.031
plot                         -0.4453      0.085     -5.219      0.000      -0.613      -0.278
funny                         0.2828      0.133      2.129      0.033       0.022       0.543
lighthearted                  0.5881      0.170      3.464      0.001       0.255       0.921
emotional                     0.3171      0.149      2.128      0.033       0.025       0.609
hopeful                       0.5598      0.165      3.401      0.001       0.237       0.883
inspiring                    -0.3120      0.159     -1.958      0.050      -0.624       0.000
reflective                    0.3406      0.141      2.410      0.016       0.064       0.618
adventurous                   1.1294      0.134      8.407      0.000       0.866       1.393
challenging                   0.8006      0.224      3.573      0.000       0.361       1.240
informative                   1.6479      0.123     13.381      0.000       1.406       1.889
mysterious                    0.2627      0.080      3.268      0.001       0.105       0.420
dark                          0.2126      0.077      2.765      0.006       0.062       0.363
author_count                 -0.1141      0.011    -10.350      0.000      -0.136      -0.092
Fiction                       0.6608      0.128      5.151      0.000       0.409       0.912
Literary                      0.2150      0.036      5.940      0.000       0.144       0.286
Social                       -0.1213      0.036     -3.386      0.001      -0.192      -0.051
Children                      0.2107      0.088      2.387      0.017       0.038       0.384
pages                     -2.557e-05      0.000     -0.220      0.826      -0.000       0.000
pages:plot                   -0.0005      0.000     -3.147      0.002      -0.001      -0.000
pages:funny                   0.0003      0.000      2.495      0.013    6.91e-05       0.001
pages:lighthearted           -0.0008      0.000     -3.862      0.000      -0.001      -0.000
relaxing                      0.8319      0.232      3.590      0.000       0.378       1.286
pages:relaxing                0.0011      0.000      2.906      0.004       0.000       0.002
tense                         0.1547      0.135      1.149      0.251      -0.109       0.419
pages:tense                   0.0007      0.000      3.482      0.001       0.000       0.001
pages:reflective             -0.0005      0.000     -3.173      0.002      

Większość zmiennych wpływa pozytywnie na zmienną objaśnianą oprócz: pages, Fantasy, Crime, Children, Romans, Realism, Fiction_character, Fiction_plot, Literary_inspiring, Literary_mix, Nonfiction_hopeful, series_emotional i lighthearted_funny.
Jednak zmienna Fantasy jest również obecna w interakcji z adventurous, a zmienna Crime w interkacji z mix.

Można wyciągnąc wniosek, że dobrze są oceniane ksiązki Fantasy jedynie, gdy mają wysoki wskaźnik przygodowości, a Crime, gdy mają mix postaci i fabuły. Gorzej są oceniane książki z genre fiction jedynie skupione na postaciach lub fabule, a nie mixie. Dodatkowo kategorie Children, Romans i Realism mają średnio niższe oceny. Również mniej lubione są książki z kategorii Literary, które są inspirujące lub mają mix fabułu i postaci, także książki Nonfiction, które są pełne nadziei i emocjonalne serie.
Zaskakująco ludzie również gorzej oceniają książki, które są śmieszne jeśli są beztroskie, możliwe, że preferowane są książki, które mają tzw. dark humor.

## Regularyzacja

$\text{Ponieważ model jest bardzo złożony (ze względu na dużą liczbę zmiennych objaśniających), a wyniki na zbiorze treningowym i walidacyjnym różnią się, w poniższej sekcji zastosowano regularyzację.}$<p>
$\text{Regularyzacja ma na celu zmniejszenie złożoności modelu, co może poprawić jego zdolność do generalizacji, a tym samym zmniejszyć ryzyko przeuczenia.}$<p>
$\text{W tym celu wykorzystane zostaną dwie metody regularyzacji: Lasso oraz Ridge.}$

$\text{W przypadku obu modeli najistotniejszym parametrem jest współczynnik regularyzacji } \lambda.$<p>
$\text{Przeprowadzona zostanie walidacja krzyżowa w celu wyboru optymalnego parametru } \lambda.$

### Lasso

In [14]:
lambdas = np.linspace(1e-10, 1e-1, 20)
for lambda_ in lambdas:
    model = Lasso(alpha=lambda_, random_state=SEED)
    train_score, validation_score = perform_cv(train_data_interactions[best_features], train_data_interactions[target], model)
    print("Lambda: {}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(lambda_, train_score, validation_score))

Lambda: 1e-10; RMSE train: 0.17892; RMSE validation: 0.18979
Lambda: 0.0052631579894736845; RMSE train: 0.23338; RMSE validation: 0.23574
Lambda: 0.01052631587894737; RMSE train: 0.24491; RMSE validation: 0.24725
Lambda: 0.015789473768421054; RMSE train: 0.25031; RMSE validation: 0.25260
Lambda: 0.02105263165789474; RMSE train: 0.25238; RMSE validation: 0.25472
Lambda: 0.026315789547368428; RMSE train: 0.25421; RMSE validation: 0.25644
Lambda: 0.03157894743684211; RMSE train: 0.25502; RMSE validation: 0.25732
Lambda: 0.036842105326315794; RMSE train: 0.25596; RMSE validation: 0.25834
Lambda: 0.04210526321578948; RMSE train: 0.25695; RMSE validation: 0.25933
Lambda: 0.04736842110526317; RMSE train: 0.25778; RMSE validation: 0.26009
Lambda: 0.052631578994736854; RMSE train: 0.25840; RMSE validation: 0.26072
Lambda: 0.05789473688421053; RMSE train: 0.25905; RMSE validation: 0.26141
Lambda: 0.0631578947736842; RMSE train: 0.25976; RMSE validation: 0.26215
Lambda: 0.0684210526631579; RMSE t

$\text{Wraz ze wzrostem parametru } \lambda \text{ wyniki na zbiorze treningowym oraz walidacyjnym ulegają pogorszeniu.}$<p>
$\text{Ponadto, dla żadnej z analizowanych wartości parametru } \lambda \text{ nie udało się uzyskać lepszych wyników niż dla zwykłej regresji liniowej.}$

### Ridge

In [15]:
lambdas = np.linspace(1e-10, 1e-1, 20)
for lambda_ in lambdas:
    model = Ridge(alpha=lambda_, random_state=SEED)
    train_score, validation_score = perform_cv(train_data_interactions[best_features], train_data_interactions[target], model)
    print("Lambda: {}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(lambda_, train_score, validation_score))

Lambda: 1e-10; RMSE train: 0.17866; RMSE validation: 0.18956
Lambda: 0.0052631579894736845; RMSE train: 0.17867; RMSE validation: 0.18953
Lambda: 0.01052631587894737; RMSE train: 0.17869; RMSE validation: 0.18951
Lambda: 0.015789473768421054; RMSE train: 0.17872; RMSE validation: 0.18951
Lambda: 0.02105263165789474; RMSE train: 0.17876; RMSE validation: 0.18951
Lambda: 0.026315789547368428; RMSE train: 0.17880; RMSE validation: 0.18952
Lambda: 0.03157894743684211; RMSE train: 0.17885; RMSE validation: 0.18954
Lambda: 0.036842105326315794; RMSE train: 0.17890; RMSE validation: 0.18956
Lambda: 0.04210526321578948; RMSE train: 0.17894; RMSE validation: 0.18958
Lambda: 0.04736842110526317; RMSE train: 0.17899; RMSE validation: 0.18961
Lambda: 0.052631578994736854; RMSE train: 0.17904; RMSE validation: 0.18963
Lambda: 0.05789473688421053; RMSE train: 0.17909; RMSE validation: 0.18966
Lambda: 0.0631578947736842; RMSE train: 0.17914; RMSE validation: 0.18969
Lambda: 0.0684210526631579; RMSE t

$\text{W przypadku modelu Ridge można zaobserwować podobną prawidłowość, jak w przypadku modelu Lasso, jeśli chodzi o spadek wyników wraz ze wzrostem parametru } \lambda.$<p>
$\text{Co prawda wyniki w okolicach } \lambda = 0.01 \text{ są nieznacznie lepsze niż dla zwykłej regresji liniowej, jednak różnica ta jest na tyle niewielka, że nie uzasadnia zastosowania regularyzacji.}$

## Zapisanie modelu

In [16]:
data_interactions=pd.read_csv("../data/data_interactions.csv")
data_interactions=data_interactions.drop(columns=['Unnamed: 0'])
train_data_interactions, test_data_interactions = train_test_split(data_interactions, test_size=0.2, random_state=SEED)
test_indices = test_data_interactions.index

#Ewaluacja modelu
model = LinearRegression(fit_intercept=True)
train_results, test_results, y_test_pred = evaluation(train_data_interactions[best_features], train_data_interactions[target], test_data_interactions[best_features], test_data_interactions[target], model)
print("Train RMSE: {}".format(round(train_results, 5)))
print("Test RMSE: {}".format(round(test_results, 5)))

#Zapisanie modelu
modelOLS = {
    "name": "OLS",
    "trainResults": train_results,
    "testResults": test_results,
    "predictions": y_test_pred,
    "indices": test_indices,
}

with open("../data/model_OLS.p", "wb") as fp:
    pickle.dump(modelOLS, fp)

Train RMSE: 0.17974
Test RMSE: 0.20202


## Podsumowanie

$\text{Najlepsze wyniki walidacji krzyżowej uzyskano dla modelu bazującego na danych z interakcjami, bez transformacji zmiennych.}$<p>
$\text{Wyniki na zbiorze treningowym (RMSE): 0.17974}$<p>
$\text{Wyniki na zbiorze testowym (RMSE): 0.20202}$<p>